# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

In [3]:
def answer_one():
    
    result = (len(spam_data[spam_data['target'] == 1]) / len(spam_data) * 100)
    return result

In [4]:
answer_one()

13.406317300789663

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    import operator

    vectorizer_total = CountVectorizer()
    vectorizer_total.fit(X_train)
    result = (sorted([(token, len(token)) for token in vectorizer_total.vocabulary_.keys()], 
                     key=operator.itemgetter(1), 
                     reverse=True)[0][0])
    return result


In [6]:
answer_two()

'com1win150ppmx3age16subscription'

In [11]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vectorizer_total = CountVectorizer()
    X_train_transformed = vectorizer_total.fit_transform(X_train)
    X_test_transformed = vectorizer_total.transform(X_test)

    model = MultinomialNB(alpha=0.1).fit(X_train_transformed, y_train)

    y_predicted = model.predict(X_test_transformed)
    result = (roc_auc_score(y_test, y_predicted))
    
    return result

In [12]:
answer_three()

0.97208121827411165

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

# incorrect
def answer_four():
    import operator

    vectorizer = TfidfVectorizer()
    X_train_transformed = vectorizer.fit_transform(X_train)

    feature_names = vectorizer.get_feature_names()
    idfs = vectorizer.idf_
    names_idfs = list(zip(feature_names, idfs))

    smallest = sorted(names_idfs, key=operator.itemgetter(1))[:20]
    smallest = pd.Series([features[1] for features in smallest], index=[features[0] for features in smallest])

    largest = sorted(names_idfs, key=operator.itemgetter(1), reverse=True)[:20]
    largest = sorted(largest, key=operator.itemgetter(0))
    largest = pd.Series([features[1] for features in largest], index=[features[0] for features in largest])
    
    result = (smallest, largest)
    return result

In [14]:
answer_four()

(to      2.198406
 you     2.265645
 the     2.707383
 in      2.890761
 and     2.976764
 is      3.003012
 me      3.111530
 for     3.206840
 it      3.222174
 my      3.231044
 call    3.297812
 your    3.300196
 of      3.319473
 have    3.354130
 that    3.408477
 on      3.463136
 now     3.465949
 can     3.545053
 are     3.560414
 so      3.566625
 dtype: float64, 000pes         8.644919
 0089           8.644919
 0121           8.644919
 01223585236    8.644919
 0125698789     8.644919
 02072069400    8.644919
 02073162414    8.644919
 02085076972    8.644919
 021            8.644919
 0430           8.644919
 07008009200    8.644919
 07099833605    8.644919
 07123456789    8.644919
 0721072        8.644919
 07753741225    8.644919
 077xxx         8.644919
 078            8.644919
 07808247860    8.644919
 07808726822    8.644919
 078498         8.644919
 dtype: float64)

In [23]:
def answer_five():
    vectorizer = TfidfVectorizer(min_df=3)
    X_train_transformed = vectorizer.fit_transform(X_train)
    X_test_transformed = vectorizer.transform(X_test)

    model = MultinomialNB(alpha=0.1).fit(X_train_transformed, y_train)
    
    y_predicted = model.predict(X_test_transformed)
    result = roc_auc_score(y_test, y_predicted)

    return result

In [22]:
answer_five()

0.94162436548223349

In [24]:
def answer_six():
    spam_data['length'] = spam_data['text'].apply(lambda x:len(x))
    result = (np.mean(spam_data['length'][spam_data['target'] == 0]), np.mean(spam_data['length'][spam_data['target'] == 1]))

    return result

In [25]:
answer_six()

(71.023626943005183, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [26]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

In [29]:
from sklearn.svm import SVC

def answer_seven():
    vectorizer = TfidfVectorizer(min_df=5)

    X_train_transformed = vectorizer.fit_transform(X_train)
    X_train_transformed_plus_length = add_feature(X_train_transformed, X_train.str.len())

    X_test_transformed = vectorizer.transform(X_test)
    X_test_transformed_plus_length = add_feature(X_test_transformed, X_test.str.len())

    model = SVC(C=10000)
    
    # train model
    model.fit(X_train_transformed_plus_length, y_train)
    
    # test model
    y_predicted = model.predict(X_test_transformed_plus_length)
    
    result = roc_auc_score(y_test, y_predicted)
    
    return result

In [30]:
answer_seven()

0.95813668234215565

In [27]:
def answer_eight():
    spam_data['length'] = spam_data['text'].apply(lambda x: len(''.join([a for a in x if a.isdigit()])))
    result = (np.mean(spam_data['length'][spam_data['target'] == 0]), np.mean(spam_data['length'][spam_data['target'] == 1]))

    return result

In [28]:
answer_eight()

(0.29927461139896372, 15.759036144578314)

In [36]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    vectorizer = TfidfVectorizer(min_df=5, ngram_range=[1, 3])

    X_train_transformed = vectorizer.fit_transform(X_train)
    X_train_transformed_plus_length = add_feature(X_train_transformed, 
                                                  [X_train.str.len(), X_train.apply(lambda x: len(''.join([a for a in x if a.isdigit()]))) ])

    X_test_transformed = vectorizer.transform(X_test)
    X_test_transformed_plus_length = add_feature(X_test_transformed, 
                                                 [X_test.str.len(), X_test.apply(lambda x: len(''.join([a for a in x if a.isdigit()]))) ])

    model = LogisticRegression(C=100)
    
    # train model
    model.fit(X_train_transformed_plus_length, y_train)
    
    # test model
    y_predicted = model.predict(X_test_transformed_plus_length)
    
    result = roc_auc_score(y_test, y_predicted)
    
    return result

In [37]:
answer_nine()

0.96533283533945646

In [38]:
def answer_ten():
    spam_data['length'] = spam_data['text'].str.findall(r'(\W)').str.len()
    result = (np.mean(spam_data['length'][spam_data['target'] == 0]), np.mean(spam_data['length'][spam_data['target'] == 1]))

    return result

In [39]:
answer_ten()

(17.291813471502589, 29.041499330655956)

In [40]:
# incorrect
def answer_eleven():
    vectorizer = TfidfVectorizer(min_df=5, analyzer='char_wb', ngram_range=[1, 3])

    X_train_transformed = vectorizer.fit_transform(X_train)
    X_train_transformed_plus_length = add_feature(X_train_transformed, 
                                                  [X_train.str.len(), X_train.apply(lambda x: len(''.join([a for a in x if a.isdigit()]))),
                                                  X_train.str.findall(r'(\W)').str.len()])


    X_test_transformed = vectorizer.transform(X_test)
    X_test_transformed_plus_length = add_feature(X_test_transformed, 
                                                 [X_test.str.len(), X_test.apply(lambda x: len(''.join([a for a in x if a.isdigit()]))),
                                                 X_test.str.findall(r'(\W)').str.len()])

    model = LogisticRegression(C=100)
    
    # train model
    model.fit(X_train_transformed_plus_length, y_train)
    
    # test model
    y_predicted = model.predict(X_test_transformed_plus_length)
    
    result = roc_auc_score(y_test, y_predicted)

    feature_names = np.array(vectorizer.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count'])
    sorted_coef_index = model.coef_[0].argsort()
    smallest = feature_names[sorted_coef_index[:10]]
    largest = feature_names[sorted_coef_index[:-11:-1]]
    
    
    return (result, list(smallest), list(largest))

In [41]:
answer_eleven()

(0.96616895574079431,
 [' i', 'i ', '.', 'ca', 'my ', ';', 'pe', ' go', 'if', 'n '],
 ['/', '0', '**', 'x', 'ww', 'uk', '***', 'mob', 'ree', '>'])